In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

from copy import deepcopy
import itertools
import pickle

import pandas as pd
from scipy.io import savemat, loadmat
import seaborn as sns
from sklearn.metrics import (log_loss, mean_squared_error, roc_curve, auc,
                             precision_recall_fscore_support, confusion_matrix)
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_validate
from tqdm import tqdm, trange, tqdm_notebook as tqdmn

from BEKML import BEMKL, plot_distplot
from utils import poly_kernel, gauss_kernel, scoring, plot_kernel_importances, plot_compare_models

sns.set(style='ticks', context='talk')
np.set_printoptions(precision=4, linewidth=100)

Populating the interactive namespace from numpy and matplotlib


/home/mtambos/anaconda/envs/pml/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data = pd.read_csv('data/wpbc.csv', names=list(range(35)), index_col=0, na_values='?')
display(data.describe())
data = data.dropna()
display(data.head())
data.describe()

,2,3,4,5,6,7,8,9,10,11,...,25,26,27,28,29,30,31,32,33,34
count,198.000000,198.000000,198.00000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,...,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,198.000000,194.000000
mean,46.732323,17.412323,22.27601,114.856566,970.040909,0.102681,0.142648,0.156243,0.086776,0.192754,...,140.347778,1404.958586,0.143921,0.365102,0.436685,0.178778,0.323404,0.090828,2.847475,3.211340
std,34.462870,3.161676,4.29829,21.383402,352.149215,0.012522,0.049898,0.070572,0.033877,0.027437,...,28.892279,586.006972,0.022004,0.163965,0.173625,0.045181,0.075161,0.021172,1.937964,5.479276
min,1.000000,10.950000,10.38000,71.900000,361.600000,0.074970,0.046050,0.023980,0.020310,0.130800,...,85.100000,508.100000,0.081910,0.051310,0.023980,0.028990,0.156500,0.055040,0.400000,0.000000
25%,14.000000,15.052500,19.41250,98.160000,702.525000,0.093900,0.110200,0.106850,0.063670,0.174075,...,118.075000,947.275000,0.129325,0.248700,0.322150,0.152650,0.275950,0.076578,1.500000,0.000000
50%,39.500000,17.290000,21.75000,113.700000,929.100000,0.101900,0.131750,0.151350,0.086075,0.189350,...,136.500000,1295.000000,0.141850,0.351300,0.402350,0.179250,0.310300,0.086890,2.500000,1.000000
75%,72.750000,19.580000,24.65500,129.650000,1193.500000,0.110975,0.172200,0.200500,0.103925,0.209325,...,159.875000,1694.250000,0.154875,0.423675,0.541050,0.207125,0.358800,0.101375,3.500000,4.000000
max,125.000000,27.220000,39.28000,182.100000,2250.000000,0.144700,0.311400,0.426800,0.201200,0.304000,...,232.200000,3903.000000,0.222600,1.058000,1.170000,0.290300,0.663800,0.207500,10.000000,27.000000


,1,2,3,4,5,6,7,8,9,10,...,25,26,27,28,29,30,31,32,33,34
0,,,,,,,,,,,,,,,,,,,,,
119513,N,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,0.07055,...,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5.0
8423,N,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,0.14710,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2.0
842517,N,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,0.08180,...,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0.0
843483,N,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,0.10520,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0.0
843584,R,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,0.10430,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0.0


,2,3,4,5,6,7,8,9,10,11,...,25,26,27,28,29,30,31,32,33,34
count,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,...,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000,194.000000
mean,46.938144,17.402320,22.300979,114.781495,969.092268,0.102774,0.142642,0.156309,0.086808,0.192885,...,140.136907,1401.756701,0.143921,0.364567,0.436010,0.178449,0.322251,0.090777,2.867526,3.211340
std,34.523646,3.171672,4.335292,21.430694,353.159959,0.012607,0.050229,0.070942,0.033962,0.027679,...,28.826843,587.040705,0.022092,0.165528,0.174757,0.045529,0.074068,0.021371,1.950588,5.479276
min,1.000000,10.950000,10.380000,71.900000,361.600000,0.074970,0.046050,0.023980,0.020310,0.130800,...,85.100000,508.100000,0.081910,0.051310,0.023980,0.028990,0.156500,0.055040,0.400000,0.000000
25%,14.250000,15.052500,19.342500,98.160000,702.525000,0.093900,0.109850,0.106075,0.063760,0.174075,...,117.925000,940.575000,0.129325,0.247550,0.322150,0.152225,0.275950,0.076368,1.500000,0.000000
50%,39.500000,17.290000,21.795000,113.700000,929.100000,0.102200,0.131750,0.152050,0.086075,0.189350,...,136.500000,1295.000000,0.141750,0.350450,0.401150,0.178500,0.310300,0.086540,2.500000,1.000000
75%,73.000000,19.580000,24.782500,129.650000,1193.500000,0.111375,0.172200,0.200500,0.103925,0.209550,...,159.875000,1694.250000,0.154450,0.423675,0.550175,0.207125,0.358475,0.101775,3.500000,4.000000
max,125.000000,27.220000,39.280000,182.100000,2250.000000,0.144700,0.311400,0.426800,0.201200,0.304000,...,232.200000,3903.000000,0.222600,1.058000,1.170000,0.290300,0.663800,0.207500,10.000000,27.000000


In [3]:
X = data.iloc[:, 1:].copy()
y = data.iloc[:, 0].copy()
N, D = X.shape
y.loc[y == 'N'] = -1
y.loc[y == 'R'] = 1
y = y.astype(int)
display(X.shape, y.shape, y.mean())

(194, 33)

(194,)

-0.5257731958762887

In [4]:
proc_data = data.copy()
proc_data.iloc[:, 0] = y
proc_data = proc_data.loc[:, list(proc_data.columns[1:]) + [proc_data.columns[0]]]
proc_data.to_csv('data/proc_wpbc.csv')
proc_data.head()

,2,3,4,5,6,7,8,9,10,11,...,26,27,28,29,30,31,32,33,34,1
0,,,,,,,,,,,,,,,,,,,,,
119513,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,0.07055,0.1865,...,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5.0,-1
8423,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,0.14710,0.2419,...,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2.0,-1
842517,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,0.08180,0.2333,...,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0.0,-1
843483,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,0.10520,0.2597,...,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0.0,-1
843584,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,0.10430,0.1809,...,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0.0,1


In [4]:
rbf_init, rbf_end = -3, 7
ply_init, ply_end = 1, 4

kernel_attrs = [('rbf', 'all', i) for i in range(rbf_init, rbf_end)]
kernels = [lambda A, B: gauss_kernel(A, B, 2**i)
           for i in range(rbf_init, rbf_end)]

kernel_attrs += [('poly', 'all', i) for i in range(ply_init, ply_end)]
kernels += [lambda A, B: poly_kernel(A, B, 1, i)
            for i in range(ply_init, ply_end)]

kernel_attrs += [('rbf', j, i) for i in range(rbf_init, rbf_end)
                 for j in range(D)]
kernels += [lambda A, B: gauss_kernel(A[:, j:j+1], B[:, j:j+1], 2**i)
            for i in range(rbf_init, rbf_end) for j in range(D)]

kernel_attrs += [('poly', j, i) for i in range(ply_init, ply_end)
                 for j in range(D)]
kernels += [lambda A, B: poly_kernel(A[:, j:j+1], B[:, j:j+1], 1, i)
            for i in range(ply_init, ply_end) for j in range(D)]
len(kernels), len(kernel_attrs)

(442, 442)

In [5]:
X_train, X_test, y_train, y_test =\
        train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

In [6]:
normalizer = Normalizer()
X_train_norm = normalizer.fit_transform(X_train)
X_test_norm = normalizer.fit_transform(X_test)

# Baseline Model

In [7]:
max_iter = 200
base_model = BEMKL(kernels=kernels, hyp_lambda_alpha=1, hyp_lambda_beta=1,
                   hyp_gamma_alpha=1, hyp_gamma_beta=1,
                   hyp_omega_alpha=1, hyp_omega_beta=1,
                   e_null_thrsh=1e-2, a_null_thrsh=1e-2,
                   filter_kernels=False, filter_sv=False, verbose=False,
                   max_iter=max_iter, hyperopt_enabled=False, calculate_bounds=False)
base_model = make_pipeline(Normalizer(), base_model)

scoring.iteration = 0
scoring.stats = []
folds = RepeatedStratifiedKFold(n_splits=4, n_repeats=5)
base_cv_results = cross_validate(base_model, X, y, cv=folds, scoring=scoring)
base_stats = deepcopy(scoring.stats)
base_cv_results, base_stats

0 - Kernels: 442/442 (1.0). SV: 142/145 (0.9793103448275862). Mean e: 0.0181. Median e: -0.0196. Std e: 0.0853. 
1 - Kernels: 442/442 (1.0). SV: 136/145 (0.9379310344827586). Mean e: 0.0148. Median e: -0.0342. Std e: 0.0982. 
2 - Kernels: 442/442 (1.0). SV: 128/146 (0.8767123287671232). Mean e: 0.0135. Median e: -0.0206. Std e: 0.0798. 
3 - Kernels: 112/442 (0.25339366515837103). SV: 127/146 (0.8698630136986302). Mean e: 0.0166. Median e: 0.0077. Std e: 0.0561. 
4 - Kernels: 432/442 (0.9773755656108597). SV: 133/145 (0.9172413793103448). Mean e: 0.0152. Median e: -0.0179. Std e: 0.0763. 
5 - Kernels: 442/442 (1.0). SV: 139/145 (0.9586206896551724). Mean e: 0.0127. Median e: -0.0223. Std e: 0.0779. 
6 - Kernels: 442/442 (1.0). SV: 128/146 (0.8767123287671232). Mean e: 0.0143. Median e: -0.0312. Std e: 0.1078. 
7 - Kernels: 432/442 (0.9773755656108597). SV: 103/146 (0.7054794520547946). Mean e: 0.0180. Median e: -0.0133. Std e: 0.0694. 
8 - Kernels: 102/442 (0.23076923076923078). SV: 144

/home/mtambos/anaconda/envs/pml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


({'fit_time': array([ 9.203 ,  7.9835,  7.6467,  7.9533,  7.5436,  8.9835,  7.7023,  7.2376,  8.0288,  9.2573,
         10.8527,  9.9686,  8.1865,  7.3853,  7.5086,  7.5116,  6.3709,  6.2813,  6.3742,  6.6055]),
  'score_time': array([0.5165, 0.487 , 0.4665, 0.4674, 0.5666, 0.5098, 0.4927, 0.5379, 0.495 , 0.4831, 0.5042,
         0.5124, 0.5048, 0.5485, 0.4848, 0.4762, 0.4737, 0.4661, 0.4623, 0.4703]),
  'test_score': array([0.7551, 0.7551, 0.8125, 0.7917, 0.8163, 0.7959, 0.7708, 0.7292, 0.7347, 0.7551, 0.7917,
         0.7917, 0.7143, 0.8163, 0.8333, 0.7708, 0.7551, 0.7551, 0.7708, 0.7292]),
  'train_score': array([0.8069, 0.7931, 0.774 , 0.7603, 0.7793, 0.7655, 0.7808, 0.8014, 0.7862, 0.7862, 0.7671,
         0.7877, 0.7793, 0.7724, 0.7671, 0.7671, 0.7724, 0.8276, 0.7945, 0.7808])},
 [{'elapsed_time': 8.923488855361938,
   'nr_kernels_used': 442,
   'nr_sv_used': 142,
   'total_kernels': 442,
   'total_sv': 145},
  {'elapsed_time': 7.687211990356445,
   'nr_kernels_used': 442,
   'nr

In [8]:
base_times = np.array([s['elapsed_time'] for s in base_stats])
base_kernels = np.array([s['nr_kernels_used'] for s in base_stats])
base_sv = np.array([s['nr_sv_used'] for s in base_stats])
print(
    f"Score: {base_cv_results['test_score'].mean()} +- {base_cv_results['test_score'].std()}\n"
    f"Time: {base_times.mean()} +- {base_times.std()}\n"
    f"Kernels: {base_kernels.mean()} +- {base_kernels.std()}\n"
    f"SVs: {base_sv.mean()} +- {base_sv.std()}"
)

Score: 0.7722363945578232 +- 0.03231630957034512
Time: 7.618676316738129 +- 1.1787609739862706
Kernels: 388.5 +- 119.04935951108683
SVs: 128.25 +- 11.755317945508747


# Kernel-sparse model

In [9]:
max_iter = 200
ksparse_model = BEMKL(kernels=kernels, hyp_lambda_alpha=1, hyp_lambda_beta=1,
                      hyp_gamma_alpha=1, hyp_gamma_beta=1,
                      hyp_omega_alpha=1e-11, hyp_omega_beta=1e9,
                      e_null_thrsh=1e-2, a_null_thrsh=1e-2,
                      filter_kernels=False, filter_sv=False, verbose=False,
                      max_iter=max_iter, hyperopt_enabled=False, calculate_bounds=False)
ksparse_pipeline = make_pipeline(Normalizer(), ksparse_model)

scoring.iteration = 0
scoring.stats = []
folds = RepeatedStratifiedKFold(n_splits=4, n_repeats=5)
ksparse_cv_results = cross_validate(ksparse_pipeline, X, y, cv=folds, scoring=scoring)
ksparse_stats = deepcopy(scoring.stats)
ksparse_cv_results, ksparse_stats

0 - Kernels: 1/442 (0.0022624434389140274). SV: 141/145 (0.9724137931034482). Mean e: 0.0067. Median e: 0.0006. Std e: 0.1259. 
1 - Kernels: 1/442 (0.0022624434389140274). SV: 140/145 (0.9655172413793104). Mean e: 0.0044. Median e: 0.0006. Std e: 0.0802. 
2 - Kernels: 2/442 (0.004524886877828055). SV: 131/146 (0.8972602739726028). Mean e: 0.0062. Median e: 0.0005. Std e: 0.1169. 
3 - Kernels: 1/442 (0.0022624434389140274). SV: 118/146 (0.8082191780821918). Mean e: 0.0060. Median e: 0.0005. Std e: 0.1087. 
4 - Kernels: 3/442 (0.006787330316742082). SV: 139/145 (0.9586206896551724). Mean e: 0.0071. Median e: 0.0006. Std e: 0.1355. 
5 - Kernels: 1/442 (0.0022624434389140274). SV: 128/145 (0.8827586206896552). Mean e: 0.0077. Median e: 0.0006. Std e: 0.1405. 
6 - Kernels: 2/442 (0.004524886877828055). SV: 127/146 (0.8698630136986302). Mean e: 0.0092. Median e: 0.0005. Std e: 0.4941. 
7 - Kernels: 1/442 (0.0022624434389140274). SV: 139/146 (0.952054794520548). Mean e: 0.0045. Median e: 0.00

/home/mtambos/anaconda/envs/pml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


({'fit_time': array([6.4306, 6.3294, 6.283 , 6.2944, 6.1326, 6.2388, 6.5526, 6.3465, 6.2336, 6.3462, 6.2089,
         6.4385, 6.3119, 6.2683, 6.2977, 6.6148, 6.5269, 6.3812, 6.4009, 6.383 ]),
  'score_time': array([0.4836, 0.4679, 0.4689, 0.4751, 0.4771, 0.5567, 0.464 , 0.4628, 0.4651, 0.4671, 0.4697,
         0.4632, 0.4711, 0.4812, 0.4674, 0.4696, 0.4868, 0.4645, 0.4765, 0.4809]),
  'test_score': array([0.7551, 0.7551, 0.8333, 0.75  , 0.7959, 0.7959, 0.75  , 0.7292, 0.7755, 0.7551, 0.75  ,
         0.7917, 0.7347, 0.7755, 0.7917, 0.7292, 0.8163, 0.7959, 0.75  , 0.7083]),
  'train_score': array([0.8069, 0.7655, 0.7671, 0.7945, 0.7724, 0.8138, 0.7808, 0.7877, 0.7724, 0.7931, 0.7808,
         0.774 , 0.7931, 0.7655, 0.7945, 0.7945, 0.7517, 0.7793, 0.7808, 0.8082])},
 [{'elapsed_time': 6.150483131408691,
   'nr_kernels_used': 1,
   'nr_sv_used': 141,
   'total_kernels': 442,
   'total_sv': 145},
  {'elapsed_time': 6.040616989135742,
   'nr_kernels_used': 1,
   'nr_sv_used': 140,
   'tota

In [10]:
ksparse_times = np.array([s['elapsed_time'] for s in ksparse_stats])
ksparse_kernels = np.array([s['nr_kernels_used'] for s in ksparse_stats])
ksparse_sv = np.array([s['nr_sv_used'] for s in ksparse_stats])
print(
    f"Score: {ksparse_cv_results['test_score'].mean()} +- {ksparse_cv_results['test_score'].std()}\n"
    f"Time: {ksparse_times.mean()} +- {ksparse_times.std()}\n"
    f"Kernels: {ksparse_kernels.mean()} +- {ksparse_kernels.std()}\n"
    f"SVs: {ksparse_sv.mean()} +- {ksparse_sv.std()}"
)

Score: 0.766921768707483 +- 0.03133090895784486
Time: 6.053864908218384 +- 0.11586270656791635
Kernels: 1.65 +- 0.792148975887743
SVs: 134.1 +- 8.245604889879214


# SV-sparse model

In [11]:
max_iter = 200
ssparse_model = BEMKL(kernels=kernels,
                      hyp_lambda_alpha=1e-11, hyp_lambda_beta=1e9,
                      hyp_gamma_alpha=1, hyp_gamma_beta=1,
                      hyp_omega_alpha=1, hyp_omega_beta=1,
                      e_null_thrsh=1e-2, a_null_thrsh=1e-2,
                      filter_kernels=False, filter_sv=False, verbose=False,
                      max_iter=max_iter, hyperopt_enabled=False, calculate_bounds=False)
ssparse_pipeline = make_pipeline(Normalizer(), ssparse_model)

scoring.iteration = 0
scoring.stats = []
folds = RepeatedStratifiedKFold(n_splits=4, n_repeats=5)
ssparse_cv_results = cross_validate(ssparse_pipeline, X, y, cv=folds, scoring=scoring)
ssparse_stats = deepcopy(scoring.stats)
ssparse_cv_results, ssparse_stats

0 - Kernels: 442/442 (1.0). SV: 127/145 (0.8758620689655172). Mean e: 0.0104. Median e: -0.0280. Std e: 0.0845. 
1 - Kernels: 102/442 (0.23076923076923078). SV: 74/145 (0.5103448275862069). Mean e: 0.0198. Median e: -0.0021. Std e: 0.0683. 
2 - Kernels: 102/442 (0.23076923076923078). SV: 54/146 (0.3698630136986301). Mean e: 0.0147. Median e: -0.0078. Std e: 0.0616. 
3 - Kernels: 442/442 (1.0). SV: 130/146 (0.8904109589041096). Mean e: 0.0160. Median e: -0.0172. Std e: 0.0720. 
4 - Kernels: 432/442 (0.9773755656108597). SV: 54/145 (0.3724137931034483). Mean e: 0.0155. Median e: -0.0174. Std e: 0.0722. 
5 - Kernels: 442/442 (1.0). SV: 116/145 (0.8). Mean e: 0.0165. Median e: -0.0212. Std e: 0.0778. 
6 - Kernels: 442/442 (1.0). SV: 79/146 (0.541095890410959). Mean e: 0.0128. Median e: -0.0257. Std e: 0.0822. 
7 - Kernels: 102/442 (0.23076923076923078). SV: 102/146 (0.6986301369863014). Mean e: 0.0150. Median e: -0.0099. Std e: 0.0670. 
8 - Kernels: 102/442 (0.23076923076923078). SV: 52/14

/home/mtambos/anaconda/envs/pml/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


({'fit_time': array([6.4598, 6.8024, 6.7456, 6.5401, 6.3233, 6.6127, 6.379 , 6.3001, 6.4754, 6.347 , 6.4128,
         6.5809, 6.5175, 6.5831, 6.3101, 6.4769, 7.0766, 6.5673, 6.4769, 6.4364]),
  'score_time': array([0.4622, 0.4639, 0.4713, 0.4614, 0.4641, 0.4652, 0.4625, 0.4659, 0.474 , 0.4719, 0.4838,
         0.4766, 0.4683, 0.4797, 0.4826, 0.469 , 0.4756, 0.4674, 0.4841, 0.4602]),
  'test_score': array([0.7551, 0.7755, 0.7917, 0.75  , 0.7959, 0.7143, 0.7917, 0.75  , 0.7551, 0.7755, 0.7708,
         0.7083, 0.7551, 0.7755, 0.7917, 0.8125, 0.7755, 0.7551, 0.75  , 0.75  ]),
  'train_score': array([0.8207, 0.7862, 0.7671, 0.7877, 0.7793, 0.7862, 0.7808, 0.8082, 0.7862, 0.7793, 0.8014,
         0.7808, 0.7862, 0.7724, 0.7603, 0.7808, 0.8138, 0.7724, 0.7877, 0.774 ])},
 [{'elapsed_time': 6.168569326400757,
   'nr_kernels_used': 442,
   'nr_sv_used': 127,
   'total_kernels': 442,
   'total_sv': 145},
  {'elapsed_time': 6.51063084602356,
   'nr_kernels_used': 102,
   'nr_sv_used': 74,
   'to

In [12]:
ssparse_times = np.array([s['elapsed_time'] for s in ssparse_stats])
ssparse_kernels = np.array([s['nr_kernels_used'] for s in ssparse_stats])
ssparse_sv = np.array([s['nr_sv_used'] for s in ssparse_stats])
print(
    f"Score: {ssparse_cv_results['test_score'].mean()} +- {ssparse_cv_results['test_score'].std()}\n"
    f"Time: {ssparse_times.mean()} +- {ssparse_times.std()}\n"
    f"Kernels: {ssparse_kernels.mean()} +- {ssparse_kernels.std()}\n"
    f"SVs: {ssparse_sv.mean()} +- {ssparse_sv.std()}"
)

Score: 0.7649659863945578 +- 0.025393226697273525
Time: 6.225758075714111 +- 0.1842578786641781
Kernels: 254.0 +- 166.0903368652132
SVs: 83.05 +- 29.9474122421287


In [13]:
import json
with open('wpbc_results.json', 'w') as fp:
    json.dump(
        {
            'ksparse': {
                'scores': list(ksparse_cv_results['test_score']),
                'times': list(ksparse_times),
                'kernels': [int(k) for k in ksparse_kernels],
                'svs': [int(s) for s in ksparse_sv],
            },
            'ssparse': {
                'scores': list(ssparse_cv_results['test_score']),
                'times': list(ssparse_times),
                'kernels': [int(k) for k in ksparse_kernels],
                'svs': [int(s) for s in ssparse_sv],
            },
            'base': {
                'scores': list(base_cv_results['test_score']),
                'times': list(base_times),
                'kernels': [int(k) for k in base_kernels],
                'svs': [int(s) for s in base_sv],
            },
            'total_kernels': len(kernels),
            'total_sv': len(X_train),
        },
        fp,
        indent=4,
        sort_keys=True
    )